# Paquetes que pueden ser de utilidad para examen

In [2]:
from __future__ import print_function, division
import pandas as pd
import pylab as pl
import os
import json
import statsmodels as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime as dt
import seaborn as sns
from pandas.tools.plotting import scatter_matrix
from scipy.stats import poisson
from scipy.stats import normal
import scipy as sp
from scipy import stats

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Polygon']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Importar bases de datos.

In [ ]:
data = pd.read_csv("https://...csv") # Recordar hacerlo siempre asegurando reproducibilidad.

#Para json, el de abajo. También es importante asegurar reproducibilidad.
json_data=open('/home/cusp/jss895/PUI2017/PUIdata/vehicle-monitoring.json').read()
data = json.loads(json_data)

#Podemos usar para enviar a PUIdata. SOLO CUANDO ESTEMOS TRABAJANDO EN COMPUTE!!!! 
os.chdir(os.environ.get("PUIDATA"))
!curl -O "https://s3.amazonaws.com/tripdata/JC-201707-citibike-tripdata.csv.zip"

#Después usamos los siguientes comandos para asegurarnos que este guardada donde corresponde
!pwd
!ls

#Una vez descargada la base de datos en la carpeta correcta, la unzippiamos:
os.system('unzip ' + PUIdata + '/nyc_pluto_16v2%20.zip -d ' + PUIdata)

#Así traemos los datos a dónde estamos trabajando
data = pd.read_csv(PUIdata + '/BORO_zip_files_csv/MN.csv')

# Manipular bases de datos.

## Eliminar columnas, filas y NaN. También convertir valores extraños en NaN. Finalmente funciones para crear nuevas columnas, formas para hacer merge y también para concatenate.

In [ ]:
# Base de datos que contiene solo las columnas deseadas. Podría crearse nuevo DataFrame 
data = data[['BBL',"DOF Number of Buildings"]] 

#Crear base de datos nueva basada en condicionales.
df = data[data["tripduration"]<5000]
df1 = df[df["gender"]=="Male"]

# Así droppiamos filas.
data.drop(['Cochice', 'Pima'])

#Así renombramos las columnas.
data.rename(columns = {'Juanito Sokoloff':"Juan"}, inplace = True)

# Así quitamos los valores que tienen NaN
data = data.dropna()

#Un drop más sofisticado, que toma cuales datos queremos quitar
#Este sirve para eliminar outliers por ejemplo
datacut = data[(df.UnitsTotal >= 10) & (df.UnitsTotal < 1000) & (df.EnergyCons > 1000)]

# Así convertimos todos los valores númericos en floats y los valores extraños en NaN.
data = data.apply(lambda x: pd.to_numeric(x, errors = "coerce"))

#Así convertimos una variable en float, suponiendo que no tiene valores raros.
data["nombrevariable"] = data["nombrevariable"].astype(float)

#Así le damos nuevos valores a los que ya estaban en una columna,por ejemplo "Female" por 2.
data.gender = data.gender.replace(2, "Female")

# Merge basados en una columna que tiene el mismo nombre en ambas bases de datos.
df = pd.merge(data, nrg, on="BBL")

#Merge utilizando columnas con mismo significado pero diferente nombre
data = pd.merge(df, df1, right_on="Country Name", left_on="Country")

#TENER CUIDADO SI QUEREMOS HACER UN INNER O UN OUTER MERGE.

#Concatenate hacia abajo. Es decir poniendo una base de datos abajo de la otra.
df1.columns = df.columns
data = pd.concat([df, df1])

#Concatenar de modo horizontal, es decir agregando una nueva columna a la base de datos.
data = pd.concat([time311, timeNYPD], axis = 1, join = "outer")



## Crear nuevas columnas y manipulación de datetime.

In [ ]:
# Así creamos columnas nuevas con operaciones matemáticas.
data['areatotal'] = data["areacuarto"] + data["areasala"] 

#Convertir columnas en datetime
data['Created Date'] = pd.to_datetime(data['Created Date'])

#Como utlizar solo una parte del datetime (En este caso conteo de la hora para hacer análisis)
time311 = df.groupby(df5["Created Date"].dt.hour).count()

# Así creamos la columna time solo con la parte de la hora, sin la fecha.
df['time'] = df['starttime'].apply(lambda x : pd.to_datetime(x).time())

# Generar función para crear nueva columna. Ejemplo uno datetime. Ejemplo para locación
def time_checker(t):
    if t > nightstart:
        return 'Night Rider'
    if t < nightend:
        return 'Night Rider'
    else:
        return 'Day Rider'

df['typee'] = df['time'].apply(lambda x: time_checker(x))


def borough(xy):
    x,y = xy
    if (x>40.698215)&(y<-73.970755):
        return "Manhattan"        
    else:
        return "Brooklyn"
    
data["borough"] = data[["start station latitude","start station longitude"]].apply(borough, axis=1)


# Descripción básica de datos

In [ ]:
# Para ver las primeras filas de la base de datos
data.head()

# Para ver todas las columnas (variables)
data.columns

# Para ver la forma
data.shape

# Para una breve descripción de media, desviación estandar, mínimo, máximo, etc.
data.describe()

#También se pueden conseguir por separado.
data["LotArea"].std()
data["LotArea"].mean()

#Usualmente esto se acompaña de un scatterplot, sección PLOTTING.

#Hacer la media para objetos dentro de una columna
data.groubby(by = "Borough").mean 

#Así contamos el número observacion
data.groupby(by = "Borough").count

# Plotting.

###  Debajo de cada gráfico hay que incluir un caption describiendo que quiere decir el gráfico.

In [ ]:
#Settear el tamaño de la figura. Si se usa para la primera queda default para las demás
fig = pl.figure(figsize(8,8)) 

# Hacer subplots
ax = fig.add_subplot(221) # Cuantos verticalmenta, Cuántos horizontalmente, Posición.
ax1 = fig.add_subplot(222)
ax2 = fig.add_subplot(223)
ax3 = fig.add_subplot(224)

# Scatter plot
ax = fig.add_subplot(111)
ax.plot(df.EnergyCons, df.UnitsTotal, '.', alpha=0.5)
ax.set_xlim(1000,1e8)
ax.set_ylim(0,1000)
ax.set_title('Total Number of Units against Total Energy Consumption per BBL')
ax.set_xlabel('Energy Consumption (kBtu)')
ax.set_ylabel('Number of Units per BBL');

#Histogramas. En este ejemplo hay varios histogramas en una sola gráfica, podríamos tomar uno solo.
#Es una buena herramienta para ver la forma de una distribución (Debe ser comprobada por tests estadísticos)
hist(df1.tripduration, bins = 20, color = "Grey", label = "Total trips")
hist(df2.tripduration, bins = 20, label = "Male")
hist(df3.tripduration, bins = 20, color="red", label = "Female")
hist(df4.tripduration, bins = 20, color="green", label = "Unknown", alpha=0.5)

plt.xlabel("Trip Duration in seconds", fontsize=12)
plt.ylabel("Amount of trips", fontsize=12)
plt.title("Trip Duration Histogram by gender", weight='bold', fontsize=14)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# Bar plot con nombres en las variables que corresponden a nombres en una columna.
# Podemos olvidar la primera parte y abajo va a gráficar basado en números.
N = len(data["Guns/100 inhabitants"])
x = range(N)
labels = [data["Country"][0], data["Country"][1], data["Country"][2], data["Country"][3], data["Country"][4], data["Country"][5], data["Country"][6], data["Country"][7], data["Country"][8], data["Country"][9], data["Country"][10], data["Country"][11], data["Country"][12], data["Country"][13], data["Country"][14], data["Country"][15], data["Country"][16], data["Country"][17], data["Country"][18], data["Country"][19], data["Country"][20], data["Country"][21]]

plt.xticks(x, labels)
plt.xticks(rotation=90)
plt.bar(x, data["Guns/100 inhabitants"], align="center")
plt.xlabel("Country", fontsize=12)
plt.ylabel("Guns per 100 inhabitants", fontsize=12)
plt.title("Guns per 100 inhabitants by country", weight='bold', fontsize=14)

#Hacer plot con línea fitteada e intervalos de confianza (MANERA FÁCIL)
sns.regplot("FirearmsPerPerson", "MassShootPerPerson", data = data)

# Hacer scatterplots con barras de errores (en este caso error = raíz cuadrada de la observación)
ax = fig.add_subplot(111)
scatter = ax.scatter (data["Average total all civilian firearms"]/1e6, data["Number of mass shootings"],color=col)
ax.errorbar(data["Average total all civilian firearms"]/1e6,data["Number of mass shootings"], xerr= np.sqrt(data["Average total all civilian firearms"]/1e6) ,
           yerr = np.sqrt(data["Number of mass shootings"] * 1.0), fmt = '.',color="black", alpha=0.3)
ax.set_ylabel('Number of mass shootings')
ax.set_xlabel("Civilian firearms (in millions)")
ax.set_ylim()
pl.show()

#Así fitteamos una Poisson
x = np.arange(poisson.ppf(0.01, k), poisson.ppf(0.99, k))
plt.plot(x, poisson.pmf(x, k), 'bo', ms=8, label='poisson pmf')
plt.vlines(x, 0, poisson.pmf(x, k), colors='orange', lw=5, alpha=0.5)

#Fittear una normal
data=random.randn(1000000)
mu, std = norm.fit(data)
x = np.linspace(-3,3, 100)
p = norm.pdf(x, 0, 1)
plt.plot(x, p, 'k', linewidth=2)
title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
plt.title(title)
plt.show()

#Scatter Matrix
from pandas.tools.plotting import scatter_matrix
scatter_matrix (data, s=300, figsize=(16, 16));
scatter_matrix?

In [9]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix?

# Plotting después de hacer modelos, básicamente para fittear líneas.

In [ ]:
# Una vez hechos los modelos podemos usar este codigo para hacer las líneas de ajuste
plt.plot(x1, l1, '.')
plt.plot(x1, linmodel1.fittedvalues, "-", color = "r")
plt.xlabel("Country", fontsize=12)
plt.ylabel("Guns per 100 inhabitants", fontsize=12)
plt.title("Guns per 100 inhabitants by country", weight='bold', fontsize=14)

#Para modelo cuadrático
pl.plot(x1, l1, '.')
pl.plot(sort(x1), sort(model2.fittedvalues), "-", color = "r")
plt.xlabel("Country", fontsize=12)
plt.ylabel("Guns per 100 inhabitants", fontsize=12)
plt.title("Guns per 100 inhabitants by country", weight='bold', fontsize=14)

#Para hacer plot de influencia
statsmodels.graphics.regressionplots.influence_plot(model, alpha  = 0.05, criterion="cooks");

### Otras formas de hacerlo, por sí las anteriores no funcionan (esta con logaritmo)

In [ ]:
w1 = lmLog.params[1]
w2 = lmLog.params[0] 

df.loc[:,"predLog"]=b1*np.log(df["LotArea"])+b2

plt.scatter(np.log(df["LotArea"]), np.log(df["AssessTot"]))
plt.plot(np.log(df["LotArea"]), df.predLog, c="k")
plt.xlabel("LogLotArea")
plt.ylabel("LogAssessTot")
plt.title("Log-log regression between LotArea and AssessTot", weight='bold', fontsize=14)

# Modelación.

In [ ]:
# Para hacer modelo líneal con una sola variable.
lm=smf.ols('AssessTot~LotArea',data=df).fit()
lm.summary()

# Mismo modelo líneal sin intercepto.
lm1 = smf.ols("AssessTot~LotArea + 0", data = df).fit()
lm1.summary()

# Para hacer modelo cuadrático
sm = smf.ols(formula='y ~ x2 + x', data=df).fit()
sm.summary()

# Modelo WLS con errores wighted.
model1 = sm.regression.linear_model.WLS(data["MassShootPerPerson"], data["FirearmsPerPerson"], 1/abs(model.resid)).fit()
model1.summary()

# Tests Estadísticos

In [ ]:
# DIFFERENCE BETWEEN SAMPLES

#Pearson
scipy.stats.pearsonr(data["tripduration"], data ["dayornight"])

#Spearman
scipy.stats.spearmanr(data["tripduration"], data ["dayornight"])

#K-S para dos muestras
scipy.stats.ks_2samp(data[(data["dayornight"] == 0)]["tripduration"],data[(data["dayornight"] == 1)]["tripduration"])

#Estadístico Z
scipy.stats.zscore(data["cualquiercolumna"])

#Estadístico T
scipy.stats.ttest_ind(a, b, axis=0, equal_var=True)

# COMPARANDO ENTRE MUESTRAS

#K-S para una muestra
scipy.stats.kstest(data["tripduration"], poisson, N=3309471)
scipy.stats.kstest(dist_p,'norm')

#K-S para dos muestras
scipy.stats.ks_2samp(data[(data["dayornight"] == 0)]["tripduration"],data[(data["dayornight"] == 1)]["tripduration"])

#Anderson Darling
scipy.stats.anderson(dist_p, dist='norm')


# GOODNESS OF FIT

#LR
-2 * (-linmodel.llf - (-seconddegreemodel.llf)))
seconddegreemodel.compare_lr_test(linmodel))

#ChiCuadrado
def chi2(data, model, errors = None):
    '''Calculates the chi sq given data, model and errors
    Arguments:
    data: series of datapoints (endogenous variable)
    model: series of predicted values corresponding to the observed data
    errors: serie of errors (optional). 
    If errors are not passes all errors are set to 1
    '''
    if errors is None:
        errors = np.ones_like(data)
    if data.shape == model.shape and data.shape == errors.shape:
        return (((data - model)**2) / errors**2).sum()
    else: 
        print ('''ERROR:
must pass arrays of identical dimension for data, model and (optional) error)''')
    return -1

## Assume that there is error in the reported energy. 
## but that is the product of two measured qusntities, each of which will have errors. 
## The minimum error is the squareroot of the value

#errors on the measured quantities
errorsnrg = np.sqrt((bblnrgdataCut['Reported Property Floor Area'])**2 +\
                (bblnrgdataCut['Site EUI(kBtu/ft2)'])**2)

# better WOULD BE to calculate each error and add in quadrature: sqrt(1/A + 1/B) which becomes
#errorsnrg = np.sqrt(1.0 / bblnrgdataCut['Reported Property Floor Area'] + 1.0 / bblnrgdataCut['Site EUI(kBtu/ft2)'])
#but this leads to insane error values (insanely small)

## Assume count statistics in the number of units as well
errorsunits = np.sqrt(bblnrgdataCut.UnitsTotal)


errorsInLogNrg = np.abs(errorsnrg / bblnrgdataCut.nrg / np.log(10))
errorsInLogUnits = np.abs(errorsunits / bblnrgdataCut.UnitsTotal / np.log(10))

bblnrgdataCut['errorsnrg'] = errorsInLogNrg
bblnrgdataCut['errorsunits'] = errorsInLogUnits

dof = min(len(bblnrgdataCut.UnitsTotal) - np.isnan(bblnrgdataCut.UnitsTotal).sum(), 
         len(bblnrgdataCut.nrg) - np.isnan(bblnrgdataCut.nrg).sum()) - 2


